In [ ]:
from lib import SyncedProp

import ipywidgets as widgets

caption = widgets.Label(value='The values of slider1 and slider2 are synchronized')
slider1, slider2 = widgets.IntSlider(description='Slider 1'),\
                    widgets.IntSlider(description='Slider 2')
display(caption, slider1, slider2)

p = SyncedProp()
p.sync_prop(slider1, 'value')
p.sync_prop(slider1, 'value')
slider2.value = 87
p.sync_prop(slider2, 'value')
assert p.value == 87
p.value = 45
assert slider1.value == slider2.value == 45

In [ ]:
p2 = SyncedProp((slider1, 'value'), (slider2, 'value'))
assert p2.value == p.value

In [ ]:
def out_content(value):
    print(f"Value of the slider: {value}")
out = widgets.interactive_output(out_content, {'value': p})
display(out)